### Extraction des moments de tchebichef des lésion en 3D (01 -> 15)

In [ ]:
from pathlib import Path
import SimpleITK as sitk
import numpy as np
import csv

data_path = Path.cwd().parent.parent / "data"

tchbichef_3d_dir_path = data_path / "15_3D_tchebichef_momentum_extraction_csv_dir"
if not tchbichef_3d_dir_path.exists():
    tchbichef_3d_dir_path.mkdir(parents = True, exist_ok = True)

# A CHOISIR:
ORDRE_CIBLE = 10 # 10 * 10 * 10 = 1000 coefficients

In [2]:
def tchebichef_polynomials(N, order):
    """Calcule les polynômes via la récurrence de Mukundan."""
    M = np.zeros((order, N))
    x = np.arange(N)
    M[0, :] = 1.0
    if order > 1:
        M[1, :] = (2.0 * x - N + 1.0) / N
    for p in range(2, order):
        term1 = ((2.0 * p - 1.0) * (2.0 * x - N + 1.0)) / (p * N)
        term2 = ((p - 1.0) / p) * (1.0 - ((p - 1.0)**2 / (N**2)))
        M[p, :] = term1 * M[p-1, :] - term2 * M[p-2, :]
    return M

def calculate_rho(N, order):
    """Constante de normalisation pour l'orthogonalité."""
    rho = np.zeros(order)
    rho[0] = N
    for p in range(1, order):
        term1 = (N**2 - p**2) / (N**2)
        term2 = (2*p - 1) / (2*p + 1)
        rho[p] = term1 * term2 * rho[p-1]
    return rho

def decompose_3d(volume, requested_order):
    """Génère le tenseur des moments T_pqr (10x10x10)."""
    D, H, W = volume.shape
    
    # Sécurité pour les dimensions plus petites que l'ordre
    o_d, o_h, o_w = min(requested_order, D), min(requested_order, H), min(requested_order, W)
    
    # Génération des matrices de polynômes
    M_d = tchebichef_polynomials(D, o_d)
    M_h = tchebichef_polynomials(H, o_h)
    M_w = tchebichef_polynomials(W, o_w)
    
    # Normalisation rho
    rho_d = calculate_rho(D, o_d)
    rho_h = calculate_rho(H, o_h)
    rho_w = calculate_rho(W, o_w)
    
    # Calcul des moments par produit tensoriel (np.einsum est le plus efficace ici)
    # T_pqr = sum_z sum_y sum_x f(z,y,x) * Mz(p,z) * My(q,y) * Mx(r,x)
    T_raw = np.einsum('zyx,pz,qy,rx->pqr', volume, M_d, M_h, M_w)
    
    # Application de la normalisation 3D
    norm_factor = 1.0 / (np.einsum('p,q,r->pqr', rho_d, rho_h, rho_w))
    T_pqr_small = T_raw * norm_factor
    
    # Remplissage du tenseur 10x10x10 final
    T_final = np.zeros((requested_order, requested_order, requested_order))
    T_final[:o_d, :o_h, :o_w] = T_pqr_small
    
    # Arrondi à 3 décimales
    return np.round(T_final, 3)

In [ ]:
"""
01 -> 15
Ce script prend environ  minutes
Extraction des moments 3D d'images médicales
"""

mslesseg_path = data_path / "01_MSLesSeg_Dataset"

# Parcours de tous les patients et leurs sous-dossiers
for mask_path in mslesseg_path.rglob("*_MASK.nii.gz"):
    arg = mask_path.stem.split("_")
    if len(arg) == 3:
        patient_id, timepoint = arg[0], arg[1]
    elif len(arg) == 2:
        patient_id, timepoint = arg[0], None

    patient_dir = mask_path.parent

    # 1. Gestion du masque 3D
    mask_sitk = sitk.ReadImage(mask_path)
    mask_bin = sitk.Cast(mask_sitk > 0, sitk.sitkUInt8)
    lesion_cc = sitk.RelabelComponent(sitk.ConnectedComponent(mask_bin))
    mask_arr = sitk.GetArrayFromImage(lesion_cc)
    num_lesions = int(mask_arr.max())

    # 2. Boucle sur les images déjà disponibles
    modality_paths = [list(patient_dir.glob("*_T1.nii.gz"))[0],
                      list(patient_dir.glob("*_T2.nii.gz"))[0],
                      list(patient_dir.glob("*_FLAIR.nii.gz"))[0]]

    for modality_path in modality_paths:
        modality_name = (modality_path.stem.split("_")[-1]).split(".")[0]
        img_arr = sitk.GetArrayFromImage(sitk.ReadImage(modality_path))

        print(f"Traitement 3D : {patient_id} {timepoint} {modality_name}: {num_lesions} lésions")

        for l_id in range(1, num_lesions + 1):
            indices = np.where(mask_arr == l_id)
            if indices[0].size < 8:  # filtre mini 2x2x2
                continue

            # Bounding box 3D
            z_min, z_max = indices[0].min(), indices[0].max()
            y_min, y_max = indices[1].min(), indices[1].max()
            x_min, x_max = indices[2].min(), indices[2].max()

            roi_vol = img_arr[z_min:z_max+1, y_min:y_max+1, x_min:x_max+1]
            roi_mask = (mask_arr[z_min:z_max+1, y_min:y_max+1, x_min:x_max+1] == l_id)
            roi_signal = roi_vol * roi_mask

            # Normalisation
            v_max = np.max(roi_signal)
            roi_norm = roi_signal / v_max if v_max > 0 else roi_signal

            # Décomposition 3D
            moments_3d = decompose_3d(roi_norm, ORDRE_CIBLE)
            flat_moments = moments_3d.flatten()

            # Sauvegarde CSV
            if len(arg) == 3:
                csv_name = f"{patient_id}_{timepoint}_{modality_name}_L{l_id}_3D_tchebichef_momentum.csv"
            elif len(arg) == 2:
                csv_name = f"{patient_id}_{modality_name}_L{l_id}_3D_tchebichef_momentum.csv"
            csv_file = tchbichef_3d_dir_path / csv_name
            with csv_file.open("w", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(flat_moments)

print("\n[TERMINÉ] Extraction 3D de tous les patients effectuée.")

Traitement 3D : P54 None T1: 5 lésions
Traitement 3D : P54 None T2: 5 lésions
Traitement 3D : P54 None FLAIR: 5 lésions
Traitement 3D : P55 None T1: 37 lésions
Traitement 3D : P55 None T2: 37 lésions
Traitement 3D : P55 None FLAIR: 37 lésions
Traitement 3D : P56 None T1: 4 lésions
Traitement 3D : P56 None T2: 4 lésions
Traitement 3D : P56 None FLAIR: 4 lésions
Traitement 3D : P57 None T1: 113 lésions
Traitement 3D : P57 None T2: 113 lésions
Traitement 3D : P57 None FLAIR: 113 lésions
Traitement 3D : P58 None T1: 43 lésions
Traitement 3D : P58 None T2: 43 lésions
Traitement 3D : P58 None FLAIR: 43 lésions
Traitement 3D : P59 None T1: 15 lésions
Traitement 3D : P59 None T2: 15 lésions
Traitement 3D : P59 None FLAIR: 15 lésions
Traitement 3D : P60 None T1: 26 lésions
Traitement 3D : P60 None T2: 26 lésions
Traitement 3D : P60 None FLAIR: 26 lésions
Traitement 3D : P61 None T1: 13 lésions
Traitement 3D : P61 None T2: 13 lésions
Traitement 3D : P61 None FLAIR: 13 lésions
Traitement 3D : P62

KeyboardInterrupt: 